# Agents From Scratch

Link: [Blog post](https://www.newsletter.swirlai.com/p/building-ai-agents-from-scratch-part)

In [1]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Literal, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl

# Visualization
import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
from rich.table import Table
from rich import box
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)

console = Console(theme=custom_theme)

In [3]:
from dataclasses import dataclass
import inspect
import os
from typing import _GenericAlias, Callable, get_type_hints
from urllib import request

import openai

### Tool Creation Utilities

In [5]:
@dataclass
class Tool:
    """A class representing a tool with a name, description, function, and parameters.

    Parameters
    ----------
    name : str
        The name of the tool
    description : str
        A description of what the tool does
    func : Callable[..., str]
        The function that implements the tool's functionality
    parameters : dict[str, dict[str, str]]
        A dictionary mapping parameter names to their descriptions and types
    """

    name: str
    description: str
    func: Callable[..., str]
    parameters: dict[str, dict[str, str]]

    def __call__(self, *args: Any, **kwds: Any) -> str:
        """Execute the tool's function with the given arguments.

        Parameters
        ----------
        *args : Any
            Positional arguments to pass to the function
        **kwds : Any
            Keyword arguments to pass to the function

        Returns
        -------
        str
            The result of executing the function
        """
        return self.func(*args, **kwds)


def parse_docstring_params(docstring: str) -> dict[str, str]:
    """Parse parameters section from a NumPy-style docstring.

    Parameters
    ----------
    docstring : str
        The docstring to parse

    Returns
    -------
    dict[str, str]
        A dictionary mapping parameter names to their descriptions
    """
    if not docstring:
        return {}

    params: dict[str, str] = {}
    lines: list[str] = docstring.split("\n")
    in_params: bool = False
    current_param: str | None = None

    for line in lines:
        line = line.strip()
        if line.startswith("Parameters:"):
            in_params = True
        elif in_params:
            if line.startswith("-") or line.startswith("*"):
                current_param = line.lstrip("- *").split(":")[0].strip()
                params[current_param] = line.lstrip("- *").split(":")[1].strip()
            elif current_param and line:
                params[current_param] += " " + line
            elif not line:
                in_params = False

    return params


def get_type_description(type_hint: Any) -> str:
    """Get a string description of a type hint.

    Parameters
    ----------
    type_hint : Any
        The type hint to describe

    Returns
    -------
    str
        A human-readable description of the type
    """
    if isinstance(type_hint, _GenericAlias):
        if type_hint._name == "Literal":
            return f"one of {type_hint.__args__}"
    return type_hint.__name__

In [6]:
# Updated version
def parse_docstring_params(docstring: str) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    This function extracts parameter descriptions from a NumPy-style docstring
    and returns them as a dictionary mapping parameter names to their descriptions.

    Parameters
    ----------
    docstring : str
        The docstring to parse, expected to be in NumPy format.

    Returns
    -------
    dict[str, str]
        A dictionary where keys are parameter names and values are their descriptions.
        Returns an empty dict if the docstring is empty or has no parameters.
    """
    if not docstring:
        return {}

    params: dict[str, str] = {}
    lines: list[str] = docstring.split("\n")
    in_params: bool = False
    current_param: str | None = None

    for line in lines:
        line = line.strip()
        if line.startswith("Parameters"):
            in_params = True
            continue
        elif in_params:
            if (
                line.startswith("Returns")
                or line.startswith("Raises")
                or line.startswith("Notes")
                or line.startswith("Examples")
            ):
                break
            if line:
                if ":" in line:
                    current_param, description = map(str.strip, line.split(":", 1))
                    # Remove leading "-", " " or "*" from the line
                    current_param = current_param.strip("-* ")
                    params[current_param] = description
                elif current_param:
                    params[current_param] += " " + line
            else:
                in_params = False

    return params

In [7]:
# Examples
# 1: Regular
def my_func1(param1: str, param2: int) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    Parameters:
    - param1: This is the first parameter.
    - param2: This is the second parameter.

    Returns:
    - A dictionary with parameter names as keys and their descriptions as values.
    """


# 2: NumPy Style
def my_func2(param1: str, param2: int) -> dict[str, str]:
    """Parse docstring parameters into a dictionary.

    Parameters
    ----------
    param1 : str
        This is the first parameter.
    param2 : int
        This is the second parameter.

    Returns
    -------
    dict[str, str]
        A dictionary with parameter names as keys and their descriptions as values.
    """


console.print(parse_docstring_params(my_func1.__doc__))
console.print(parse_docstring_params(my_func2.__doc__))

{'param1': 'This is the first parameter.', 'param2': 'This is the second parameter.'}

{'param1': 'str This is the first parameter.', 'param2': 'int This is the second parameter.'}

In [8]:
type_hints_dict: dict[str, Any] = get_type_hints(my_func2)
print(type_hints_dict)

print(get_type_description(type_hints_dict.get("param1")))

{'param1': <class 'str'>, 'param2': <class 'int'>, 'return': dict[str, str]}
str


<br>

### Tool Creation Decorator

In [9]:
console.print(inspect.getdoc(my_func1))

print("===== OR =====\n\n")

# OR
print(my_func1.__doc__)

Parse docstring parameters into a dictionary.

Parameters:
- param1: This is the first parameter.
- param2: This is the second parameter.

Returns:
- A dictionary with parameter names as keys and their descriptions as values.

===== OR =====


Parse docstring parameters into a dictionary.

    Parameters:
    - param1: This is the first parameter.
    - param2: This is the second parameter.

    Returns:
    - A dictionary with parameter names as keys and their descriptions as values.
    


In [10]:
inspect.signature(my_func2)

<Signature (param1: str, param2: int) -> dict[str, str]>

In [12]:
def tool(name: str | None = None) -> str:
    """Decorator function to create a Tool object from a function.

    Parameters
    ----------
    name : str | None, optional
        Custom name for the tool. If None, uses the function name, by default None

    Returns
    -------
    Callable
        Decorator function that creates a Tool object
    """

    def decorator(func: Callable[..., str]) -> Tool:
        """Inner decorator function that processes the decorated function.

        Parameters
        ----------
        func : Callable[..., str]
            Function to be converted into a Tool object

        Returns
        -------
        Tool
            Tool object created from the decorated function
        """
        tool_name: str = name or func.__name__
        description: str = inspect.getdoc(func) or "No description provided."

        type_hints: dict[str, Any] = get_type_hints(func)
        param_docs: dict[str, str] = parse_docstring_params(description)
        sig: inspect.Signature = inspect.signature(func)
        params: dict[str, Any] = {}

        for param_name, param in sig.parameters.items():
            params[param_name] = {
                "type": get_type_description(type_hints.get(param_name)),
                "description": param_docs.get(param_name, "No description provided."),
            }
        return Tool(
            name=tool_name,
            description=description,
            func=func,
            parameters=params,
        )

    return decorator

### Create Currency Convertion Tool

In [ ]:
@tool()
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    pass